# Команда Даниссимо, база данных "Мониторинг вузов"

## Создание дополнительных переменных и удаление выбросов

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_excel('modified_datasets\\result_2021_analized.xlsx') # В датасете созданно новоле поле Research, полученное из рейтинга THE

In [536]:
df = df[df['Общая численность студентов, обучающихся по программам бакалавриата, специалитета, магистратуры'] >= 1000]

In [537]:
# Добаление новых переменных
df['Число НПР'] = df['Общая численность научных работников (без внешних совместителей и работающих по договорам ГПХ)'] + df['Общая численность ППС (без внешних совместителей и работающих по договорам ГПХ)']
df['Число НИОКР в рассчете на 100 НПР'] = df['Общий объем научно-исследовательских и опытно-конструкторских работ (далее – НИОКР)'] / df['Число НПР'] * 100
df['Вклад вуза в общую сумму НИОКР'] = df['Число НИОКР в рассчете на 100 НПР'] / df['Число НИОКР в рассчете на 100 НПР'].sum()
df['Число статей, подготовленных совместно с зарубежными организациями на 100 НПР']= df['Число статей, подготовленных совместно с зарубежными организациями']/df['Число НПР'] * 100
df['Численность зарубежных ведущих профессоров, преподавателей и исследователей, работающих в образовательной организации не менее 1 семестра на 100 НПР']= df['Численность зарубежных ведущих профессоров, преподавателей и исследователей, работающих (работавших) в образовательной организации не менее 1 семестра']/df['Число НПР']*100

In [538]:
# Столбцы, которые будут учитываться при построении модели
used_feachers = [
    'Количество цитирований публикаций, изданных за последние 5 лет, индексируемых в информационно-аналитической системе научного цитирования Scopus в расчете на 100 НПР',
    'Количество центров коллективного пользования научным оборудованием',
    'Число статей, подготовленных совместно с зарубежными организациями на 100 НПР',
    'Численность зарубежных ведущих профессоров, преподавателей и исследователей, работающих в образовательной организации не менее 1 семестра на 100 НПР',
    'Число НПР, имеющих ученую степень кандидата и доктора наук, в расчете на 100 студентов',
    'Research2',

]

In [539]:
df = df[used_feachers].dropna() # Удаление вузов, у которых неизвестно значение столбца Research

In [540]:
df

,"Количество цитирований публикаций, изданных за последние 5 лет, индексируемых в информационно-аналитической системе научного цитирования Scopus в расчете на 100 НПР",Количество центров коллективного пользования научным оборудованием,"Число статей, подготовленных совместно с зарубежными организациями на 100 НПР","Численность зарубежных ведущих профессоров, преподавателей и исследователей, работающих в образовательной организации не менее 1 семестра на 100 НПР","Число НПР, имеющих ученую степень кандидата и доктора наук, в расчете на 100 студентов",Research2
41,3840.74,1,150.583090,22.448980,8.56,46.8
42,571.26,3,7.013575,1.809955,6.18,13.5
43,241.14,1,11.662818,0.923788,6.41,10.6
44,965.91,4,23.497997,1.268358,14.28,11.2
45,762.37,2,12.598425,4.724409,5.04,11.4
...,...,...,...,...,...,...
436,389.02,1,4.773562,2.203182,3.62,9.6
442,301.59,0,6.204380,0.000000,8.00,8.3
443,938.50,4,9.432049,1.217039,4.45,13.0
463,362.81,3,21.102151,1.612903,6.34,18.4


In [541]:
# Столбцы которые необходимо нормировать
to_norm_feachers = ['Количество цитирований публикаций, изданных за последние 5 лет, индексируемых в информационно-аналитической системе научного цитирования Scopus в расчете на 100 НПР',
    'Количество центров коллективного пользования научным оборудованием',
    'Число статей, подготовленных совместно с зарубежными организациями на 100 НПР',
    'Численность зарубежных ведущих профессоров, преподавателей и исследователей, работающих в образовательной организации не менее 1 семестра на 100 НПР',
    ]

![Формула нормирования](https://dev-gang.ru/static/storage/37403660903642076772234929412457116574.png)

In [542]:
# Нормализация по формуле
for i in to_norm_feachers:
    feat = df[i]
    df[i] = (feat-feat.min()) / (feat.max() - feat.min())

## Построение модели линейной регрессии

In [543]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

![Уравнение линейной энергии](https://wikimedia.org/api/rest_v1/media/math/render/svg/5db3071becf5c09f0db5a7f113e57c1184973a17)

In [544]:
x_train, x_test, y_train, y_test = train_test_split(df.drop('Research2', axis=1), df['Research2'], test_size=0.00001, random_state=1) #

In [545]:
regressor = LinearRegression()
regressor.fit(x_train, y_train)

LinearRegression()

In [546]:
coefs = regressor.coef_ # Печать весов
for i in range(len(used_feachers) - 1):
    print(used_feachers[i][:30], coefs[i])
print(regressor.intercept_) # печать свободного члена

Количество цитирований публика 17.240245118278104
Количество центров коллективно 11.272808692221211
Число статей, подготовленных с 5.4592582295917325
Численность зарубежных ведущих 19.049101216343225
Число НПР, имеющих ученую степ 1.2502656975313506
1.1169365864405183


## Проверка статистической значимости весов

In [547]:
import statsmodels.api as sm

In [548]:
X = x_train
y = y_train

In [549]:
X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary()) # Получение сводки

                            OLS Regression Results                            
Dep. Variable:              Research2   R-squared:                       0.594
Model:                            OLS   Adj. R-squared:                  0.564
Method:                 Least Squares   F-statistic:                     20.16
Date:                Tue, 20 Dec 2022   Prob (F-statistic):           2.44e-12
Time:                        16:08:24   Log-Likelihood:                -254.14
No. Observations:                  75   AIC:                             520.3
Df Residuals:                      69   BIC:                             534.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                                                                                                                                                           coef    std err          t      P>|t|  